### 기능 3: 뉴스 유사도 기반 패턴 매칭 지표 Refined

24시간동안 최고 상승 변동, 최고 하락 변동 정보를 보여주는걸로 바꿈 

차트 데이터를 안보여주는 상태에서는 이게 더 깔끔하다고 판단 

In [1]:
#!pip install openai
#!pip install ccxt 
#!pip install faiss-cpu

In [1]:
import numpy as np 
import pandas as pd 
import os 
import torch.nn as nn 
import torch.nn.functional as F  
import torch 
import math 
import faiss # pip install faiss-cpu 
from datetime import datetime, timedelta 
import json 
from tokenization_roberta_spm import FairSeqRobertaSentencePieceTokenizer
from transformers import * 
import ccxt
import openai

In [2]:
openai.api_key = "<MASKED>" 

In [3]:
# load candidate embeddings 
candidate_embeddings = np.load("candidate_embeddings_full.npy") 

candidate_embeddings.shape

(109280, 768)

In [4]:
# load news dataframe 
news_df = pd.read_csv("coinness_full.csv") 

news_df.shape

(109280, 3)

In [5]:
news_df.head(3) 

,titles,contents,datetimes
0,"TUSD/USDT 페어, 오늘 최대 80센트까지 디페깅 심화",코인데스크가 트레이딩뷰 데이터를 인용해 바이낸스US 기준 TUSD/USDT 페어가 ...,2023-06-28 15:34:00
1,"BFC, 블록체인 게임 플랫폼 젬허브와 파트너십",바이프로스트(BFC)가 공식 미디엄을 통해 블록체인 게임 플랫폼 젬허브(GemHUB...,2023-06-28 15:31:00
2,"삭제 됐다던 탈중앙화 SNS 다무스, 앱스토어서 v1.5 출시",앱스토어에서 전날 삭제된 것으로 알려진 탈중앙화 소셜 네트워크 프로토콜 노스트(No...,2023-06-28 15:29:00


In [6]:
titles = news_df["titles"].values 
contents = news_df["contents"].values 
published_datetimes = news_df["datetimes"].values 

candidate_texts = [] 
for i in range(len(contents)):
    s = str(titles[i]) + "\n" + str(contents[i]) 
    candidate_texts.append(s) 

In [7]:
tokenizer = FairSeqRobertaSentencePieceTokenizer.from_pretrained("fairseq-roberta-all-model") 
embedding_model = AutoModel.from_pretrained("LDKSolutions/KR-cryptoroberta-base")
config = AutoConfig.from_pretrained("LDKSolutions/KR-cryptoroberta-base") 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
embedding_model.to(device)
embedding_model.eval() 
print()

Didn't find file fairseq-roberta-all-model/added_tokens.json. We won't load it.
Didn't find file fairseq-roberta-all-model/special_tokens_map.json. We won't load it.
Didn't find file fairseq-roberta-all-model/tokenizer_config.json. We won't load it.
loading file fairseq-roberta-all-model/spm.model
loading file fairseq-roberta-all-model/dict.txt
loading file None
loading file None
loading file None
loading configuration file fairseq-roberta-all-model/config.json
Model config RobertaConfig {
  "_name_or_path": "fairseq-roberta-all-model",
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 51

[PASS] spm_id: madeupword0000 | fairseq_id: 51998
[PASS] spm_id: madeupword0001 | fairseq_id: 51999


Some weights of the model checkpoint at LDKSolutions/KR-cryptoroberta-base were not used when initializing XLMRobertaModel: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaModel were not initialized from the model checkpoint at LDKSolutions/KR-cryptoroberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions

In [8]:
sentiment_model = XLMRobertaForSequenceClassification.from_pretrained("LDKSolutions/KR-cryptoroberta-base") 
sentiment_model.to(device) 
sentiment_model.eval() 
print()

loading configuration file https://huggingface.co/LDKSolutions/KR-cryptoroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/5acbd794ed75d62f6347bda79a2a0adc5451fefc92468deadecb8cac7e4d0be1.36fa43760e5e542e4e396ae3c952a2088b8b9d1fe91d1063514191de83206874
Model config XLMRobertaConfig {
  "_name_or_path": "fairseq-roberta-all-model/checkpoint_last.pt",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers"

In [9]:
# calculate news sentiment score
def get_sentiment_score(text): 
    encoded_inputs = tokenizer(text, max_length=512, padding="max_length", truncation=True, return_tensors="pt").to(device) 
    with torch.no_grad(): 
        sentiment = sentiment_model(**encoded_inputs)[0] 
        sentiment = nn.Softmax(dim=1)(sentiment)[0] 
    return sentiment 

In [10]:
# 후보 뉴스 문서들의 임베딩을 이용해서 인덱스를 FAISS를 사용해서 빌드한다. 
# 인덱스를 빌드하는 이유는 빠른 검색을 위해서다. 
use_cosine_sim = True # using cosine similarity 
if use_cosine_sim: 
    index = faiss.IndexIDMap2(faiss.IndexFlatIP(config.hidden_size)) 
    faiss.normalize_L2(candidate_embeddings.astype(np.float32)) 
else: # defaults to L2 
    index = faiss.IndexIDMap2(faiss.IndexFlatL2(config.hidden_size)) 

index.add_with_ids(candidate_embeddings.astype(np.float32), np.array(range(0, len(candidate_embeddings)), dtype=int)) 
index.nprobe = 64 # hyperparameter 

In [11]:
def convert_json_chart_data_to_pd(json_file:str):
    with open(json_file) as f: 
        d = json.load(f) 
    chart_df = pd.DataFrame(d) 
    chart_df = chart_df.rename(columns={0:"timestamp", 1:"open", 2:"high", 3:"low", 4:"close", 5:"volume"})
    binance = ccxt.binance() 
    dates = chart_df["timestamp"].values 
    timestamp = [] 
    for i in range(len(dates)): 
        date_string = binance.iso8601(int(dates[i])) 
        date_string = date_string[:10] + " " + date_string[11:-5]
        timestamp.append(date_string) 
    chart_df["datetime"] = timestamp 
    chart_df.drop(columns={"timestamp"}, inplace=True) 
    chart_df.set_index(pd.DatetimeIndex(chart_df["datetime"]), inplace=True) 
    return chart_df 

In [12]:
# read 30m BTCUSDT chart data 
df30m = convert_json_chart_data_to_pd("btcusdt_binance_30m_20230829.json")

In [13]:
df30m.head()

,open,high,low,close,volume,datetime
datetime,,,,,,
2017-08-17 04:00:00,4261.48,4280.56,4261.32,4261.45,11.308926,2017-08-17 04:00:00
2017-08-17 04:30:00,4280.00,4313.62,4267.99,4308.83,35.872083,2017-08-17 04:30:00
2017-08-17 05:00:00,4308.83,4328.69,4304.31,4320.00,21.048648,2017-08-17 05:00:00
2017-08-17 05:30:00,4320.00,4320.00,4291.37,4315.32,2.186268,2017-08-17 05:30:00
2017-08-17 06:00:00,4330.29,4330.29,4309.37,4311.02,3.566277,2017-08-17 06:00:00


In [14]:
# 검색이 최적화 되지 않아서 느리긴 하지만 지금 단계에서는 상관 없을듯 
def get_relevant_chart_segment(chart_df, datestr): 
    df30m_idx = -1 
    cur_date = chart_df["datetime"].values 
    news_datestr = datetime.strptime(datestr, "%Y-%m-%d %H:%M:%S") 
    for i in range(len(cur_date)-1): 
        current_date = datetime.strptime(cur_date[i], "%Y-%m-%d %H:%M:%S") 
        next_date = datetime.strptime(cur_date[i+1], "%Y-%m-%d %H:%M:%S") 
        if news_datestr >= current_date and news_datestr < next_date:
            df30m_idx = i 
            break 
    return df30m_idx 

In [15]:
def get_ranks(query_title, query_content, K=5):
    encoded_query = tokenizer(str(query_title), str(query_content), max_length=512, truncation=True, padding="max_length", return_tensors="pt").to(device)
    query_embedding = embedding_model(**encoded_query)[0][:, 0, :].detach().cpu().numpy() 

    distances, indices = index.search(query_embedding, 1000) 
    topK = K # MVP로는 탑 5개만 보여줌  

    print("query news")
    query_text = query_title + "\n" + query_content 
    print(query_text) 
    sentiment_scores = get_sentiment_score(query_text) 
    sentiment_scores = sentiment_scores.detach().cpu().numpy()
    positive = round(sentiment_scores[0] * 100)  
    negative = round(sentiment_scores[1] * 100) 
    neutral = round(sentiment_scores[2] * 100) 
    print(f"*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: {positive}% | 악재: {negative}% | 중립: {neutral}% ***") 

    for _ in range(5): # 공백 출력 
        print() 

    top5_news, top5_max_high, top5_min_low = [], [], [] 

    cnt = 0 
    for i in range(len(indices[0][:topK+3])): 
        if candidate_texts[indices[0][i]] == query_text:
            continue
        if cnt == topK: 
            break 
        print(f"Rank {cnt+1}") 
        print() 
        print(candidate_texts[indices[0][i]])  
        top5_news.append(candidate_texts[indices[0][i]]) 
        print(f"날짜: {published_datetimes[indices[0][i]]}") 
        sentiment_scores = get_sentiment_score(candidate_texts[indices[0][i]]) 
        sentiment_scores = sentiment_scores.detach().cpu().numpy() 
        positive = round(sentiment_scores[0] * 100)  
        negative = round(sentiment_scores[1] * 100) 
        neutral = round(sentiment_scores[2] * 100) 
        print(f"*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: {positive}% | 악재: {negative}% | 중립: {neutral}% ***")
        cnt += 1 

        # calculate 1hr, 4hrs, 8hrs, 12hrs, 24hrs volatility 
        news_idx = get_relevant_chart_segment(df30m, published_datetimes[indices[0][i]]) 

        if news_idx != -1: 
            close = df30m["close"].values[news_idx] 
            max_high = max(df30m["high"].values[news_idx:news_idx+48]) # next 24 hours 
            min_low = min(df30m["low"].values[news_idx:news_idx+48]) 
            delta_high = round((max_high - close)/close * 100, 4)  
            delta_low = round((min_low - close)/close * 100, 4) 
            top5_max_high.append(delta_high) 
            top5_min_low.append(delta_low)
            if delta_high >= 0:
                delta_high = str(delta_high) 
                delta_high = "+" + delta_high
            if delta_low >= 0: 
                delta_low = str(delta_low) 
                delta_low = "+" + delta_low 
            print(f"24시간동안 최고 상승치: {delta_high}% | 24시간동안 최고 하락치: {delta_low}%") 
        else: 
            print("해당 뉴스가 나온 시점의 차트 데이터가 존재하지 않습니다! 다른 차트 데이터 소스를 참고해보세요.") 

        print("=" * 100) 
        print()
        print() 
    
    mu_high = np.mean(top5_max_high) 
    mu_low = np.mean(top5_min_low) 
    
    print(f"유사뉴스들이 나온 시점의 24시간동안 평균 비트코인 가격 최고 상승치: {mu_high}%") 
    print(f"유사뉴스들이 나온 시점의 24시간동안 평균 비트코인 가격 최고 하락치: {mu_low}%") 
    
    
    print("GPT-4가 결과 분석중...기다려주세요...") 

    query = f"쿼리 뉴스와, 해당 쿼리 뉴스와 유사하다는 과거 뉴스 5개가 유사도 순서로 랭킹이 되었어. 이때 쿼리 뉴스와 랭크된 뉴스 사이에 어떤점들이 유사한지 알기 쉽게 설명해줘. 그리고 쿼리와 유사한 뉴스들이 나온 시점부터 24시간동안 평균 비트코인 갸격 최고 상승치와 최고 하락치가 주어졌을때 이 변동성과 뉴스의 내용과 관련해서 패턴이 있는지 분석해줘. \n query:{query_text} \n rank1:{top5_news[0]} \n rank2:{top5_news[1]} \n rank3:{top5_news[2]} \n rank4:{top5_news[3]} \n rank5:{top5_news[4]} \n 유사뉴스들이 나온 시점의 24시간동안 평균 비트코인 가격 최고 상승치: {mu_high}% \n 유사뉴스들이 나온 시점의 24시간동안 평균 비트코인 가격 최고 하락치: {mu_low}%"
    
    
    response = openai.ChatCompletion.create(
        model = "gpt-4", 
        messages = [
            {"role":"user", "content":query}
        ], 
        temperature=0.5 
    )

    gpt_response = response["choices"][0]["message"]["content"] 

    print("*** GPT-4가 쿼리 뉴스와 가장 유사한 상위 5개의 뉴스와 각 유사 뉴스가 나온 시점 이후의 24시간동안의 비트코인 가격의 최고 상승과 하락 변동성을 분석한 결과 ***")
    print()
    print(gpt_response) 

## Example Run 1 

In [16]:
query_title = "크립토 '공포·탐욕 지수' 49...투심 대폭 개선"
query_content = """암호화폐 데이터 제공 업체 알터너티브(Alternative)의 자체 추산 '공포·탐욕 지수'가 전날 대비 10포인트 오른 49를 기록했다. 투심이 대폭 개선됐으며 중립 단계로 전환 됐다. 해당 지수는 0에 가까울수록 시장의 극단적 공포를 나타내며, 100에 가까울수록 극단적 낙관을 의미한다. 공포 탐욕 지수는 변동성(25%), 거래량(25%), SNS 언급량(15%), 설문조사(15%), 비트코인 시총 비중(10%), 구글 검색량(10%) 등을 기준으로 산출된다."""


get_ranks(query_title, query_content) 

query news
크립토 '공포·탐욕 지수' 49...투심 대폭 개선
암호화폐 데이터 제공 업체 알터너티브(Alternative)의 자체 추산 '공포·탐욕 지수'가 전날 대비 10포인트 오른 49를 기록했다. 투심이 대폭 개선됐으며 중립 단계로 전환 됐다. 해당 지수는 0에 가까울수록 시장의 극단적 공포를 나타내며, 100에 가까울수록 극단적 낙관을 의미한다. 공포 탐욕 지수는 변동성(25%), 거래량(25%), SNS 언급량(15%), 설문조사(15%), 비트코인 시총 비중(10%), 구글 검색량(10%) 등을 기준으로 산출된다.
*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: 71% | 악재: 23% | 중립: 5% ***





Rank 1

크립토 '공포·탐욕 지수' 43...투심 소폭 개선
암호화폐 데이터 제공 업체 알터너티브(Alternative)의 자체 추산 '공포·탐욕 지수'가 전날(39)보다 4포인트 오른 43을 기록했다. 투심이 소폭 개선됐음을 나타낸다.  해당 지수는 0에 가까울수록 시장의 극단적 공포를 나타내며, 100에 가까울수록 극단적 낙관을 의미한다. 공포 탐욕 지수는 유동성(25%), 거래량(25%), SNS 언급량(15%), 설문조사(15%), 비트코인 시총 비중(10%), 구글 검색량(10%) 등을 기준으로 산출된다.
날짜: 2020-09-23 13:28:00
*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: 62% | 악재: 31% | 중립: 7% ***
24시간동안 최고 상승치: +0.2249% | 24시간동안 최고 하락치: -3.5829%


Rank 2

크립토 '공포·탐욕 지수' 39...투심 제자리
암호화폐 데이터 제공 업체 알터너티브(Alternative)의 자체 추산 '공포·탐욕 지수'가 전날보다 1 포인트 내린 39를 기록했다. 투자 심리가 전날과 비슷한 수준으로 유지됐다. 해당 지수는 0에 가까울수록 시장의 극단적 공포를 나타내며, 100에 가까울수록 극단적 낙관을 의미한다. 공포 탐

# Example Run 2

In [17]:
query_title = """FT \"온타리오 연기금, 암호화폐 투자 중단...FTX 사태 손실 여파\"""" 
query_content = """파이낸셜타임즈가 FTX 파산 사태로 9500만 달러 상당의 손실을 입은 온타리오 교사연금(OTPP)이 손실 전액을 상각 처리한 뒤 암호화폐 투자에서 손을 떼기로 했다고 보도했다. OTPP 이사장 조 테일러(Jo Taylor)는 "우리는 FTX 사태로 인한 피해 등에 대해 여전히 내부 조사를 진행하고 있다. 또 연금 가입자들의 피드백을 고려했을 때, 다시 암호화폐 투자에 나서는 것은 현명한 처사가 아니라고 생각한다. 아울러 FTX 사태 이전에 우리는 매우 신중히 실사를 진행한 끝에 투자를 단행했지만 기대했던 결과를 얻지 못했다"고 설명했다."""

get_ranks(query_title, query_content) 

query news
FT "온타리오 연기금, 암호화폐 투자 중단...FTX 사태 손실 여파"
파이낸셜타임즈가 FTX 파산 사태로 9500만 달러 상당의 손실을 입은 온타리오 교사연금(OTPP)이 손실 전액을 상각 처리한 뒤 암호화폐 투자에서 손을 떼기로 했다고 보도했다. OTPP 이사장 조 테일러(Jo Taylor)는 "우리는 FTX 사태로 인한 피해 등에 대해 여전히 내부 조사를 진행하고 있다. 또 연금 가입자들의 피드백을 고려했을 때, 다시 암호화폐 투자에 나서는 것은 현명한 처사가 아니라고 생각한다. 아울러 FTX 사태 이전에 우리는 매우 신중히 실사를 진행한 끝에 투자를 단행했지만 기대했던 결과를 얻지 못했다"고 설명했다.
*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: 1% | 악재: 10% | 중립: 90% ***





Rank 1

포브스 "FTX 회계 감사 업체 아르마니노, 암호화폐 준비금 감사 서비스 종료"
포브스에 따르면 FTX 회계 감사 업체 아르마니노(Armanino)가 암호화폐 업체에 대한 준비금 증명 감사 서비스를 종료하기로 결정한 것으로 알려졌다. 업계 관계자는 포브스에 "회사 평판 리스크가 커지면서, 아르마니노가 이 같은 결정을 내렸다. 아직 명확한 회계 기준이 없는 암호화폐 업계 고객을 받아들이는 것을 주저하게 됐다"고 설명했다. 아르마니노는 지난 2014년부터 암호화폐 업체 회계 감사 서비스를 시작했으며, 이후 FTX를 포함 넥소, 크라켄, 게이트 아이오 등의 준비금 증명 감사를 맡아왔다. 그러나 최근 FTX의 파산 후 집단 소송에 직면하는 등 어려움을 겪고 있다.
날짜: 2022-12-16 02:03:00
*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: 1% | 악재: 13% | 중립: 86% ***
24시간동안 최고 상승치: +0.6844% | 24시간동안 최고 하락치: -5.0839%


Rank 2

서클, 페이 서비스 지원 전면 중단
골드만삭스가 지원하는 암호화폐 기업 서클이 자사의 암호화폐

# Example Run 3

In [18]:
query_title = "BTC 3년 이상 보유 장기 홀더 비중 40%... 사상 최고" 
query_content = """코인데스크에 따르면 비트파이넥스 애널리스트들은 3년 이상 BTC를 이동시키지 않은 장기 홀더 비중이 40%로 사상 최고에 달했다고 밝혔다. 이들은 장기 홀더 순포지션 변화를 자세히 보면 오래 3월 이후 일관된 매집 추세를 보였다. 그러나 강세장, 약세장 모두를 견뎌낸 홀더들과는 달리 이번 약세장에서 코인을 매수한 신규 장기 홀더들은 불안감을 내비치고 있다. 이들은 7월 BTC가 $29,000선에서 하락하는 동안 매도에 나섰다고 설명했다."""
get_ranks(query_title, query_content) 

query news
BTC 3년 이상 보유 장기 홀더 비중 40%... 사상 최고
코인데스크에 따르면 비트파이넥스 애널리스트들은 3년 이상 BTC를 이동시키지 않은 장기 홀더 비중이 40%로 사상 최고에 달했다고 밝혔다. 이들은 장기 홀더 순포지션 변화를 자세히 보면 오래 3월 이후 일관된 매집 추세를 보였다. 그러나 강세장, 약세장 모두를 견뎌낸 홀더들과는 달리 이번 약세장에서 코인을 매수한 신규 장기 홀더들은 불안감을 내비치고 있다. 이들은 7월 BTC가 $29,000선에서 하락하는 동안 매도에 나섰다고 설명했다.
*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: 94% | 악재: 2% | 중립: 5% ***





Rank 1

비트와이즈 CEO "BTC, 어려울 때 강했다"
암호화폐 전문 미디어 AMB크립토에 따르면, 암호화폐 전문 투자기관 비트와이즈(Bitwise)의 최고경영자이자 전 페이스북 제품 매니저 헌터 호슬리(Hunter Horsley)가 “미국 뉴욕증시 스탠더드앤드푸어스(S&P)500 지수가 이날 12% 가까이 떨어졌다. 지난 100년 간 이 같은 적은 딱 두 번 있었다. 한 번은 1929년, 한 번은 1987년이었다. 반면 BTC는 안정세를 유지했다. 역사적 폭락장에서 매번 BTC는 강세를 보였다”고 덧붙였다.
날짜: 2020-03-17 09:57:00
*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: 90% | 악재: 5% | 중립: 5% ***
24시간동안 최고 상승치: +5.6805% | 24시간동안 최고 하락치: -3.386%


Rank 2

ETH 고래 주소 수 3년래 최저.. 개인투자자는 증가세
크립토포테이토에 따르면 개인투자자들은 이더리움을 계속해서 매입하는 반면, 이더리움 고래(대형투자자) 수는 3년래 최저치로 떨어졌다. 글래스노드에 따르면 0 ETH 초과 보유 주소 수는 2018년부터 빠르게 증가하기 시작해 1000만개 미만에서 5500만개 이상으로 급증했다. 0.1 ETH 이상 보유 주소 수도 약

# Example Run 4

In [19]:
query_title="트위터, 암호화폐 결제 서비스 라이선스 취득"
query_content="""핀볼드가 미국 금융 라이선스 정보 플랫폼 NMLS를 인용해 X(구 트위터)가 미국 로드아일랜드에서 통화 송금업자(Currency Transmitter) 라이선스를 취득했다고 전했다. 이 라이선스는 가상자산 결제 및 거래에 필요하며 암호화폐 거래소, 월렛, 결제업체 등이 보유하고 있다. 미디어는 “X는 해당 라이선스 취득으로 전세계 수백만 이용자에게 암호화폐 전송 및 결제 서비스를 제공할 수 있게 됐다”고 설명했다."""

get_ranks(query_title, query_content) 

query news
트위터, 암호화폐 결제 서비스 라이선스 취득
핀볼드가 미국 금융 라이선스 정보 플랫폼 NMLS를 인용해 X(구 트위터)가 미국 로드아일랜드에서 통화 송금업자(Currency Transmitter) 라이선스를 취득했다고 전했다. 이 라이선스는 가상자산 결제 및 거래에 필요하며 암호화폐 거래소, 월렛, 결제업체 등이 보유하고 있다. 미디어는 “X는 해당 라이선스 취득으로 전세계 수백만 이용자에게 암호화폐 전송 및 결제 서비스를 제공할 수 있게 됐다”고 설명했다.
*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: 2% | 악재: 0% | 중립: 98% ***





Rank 1

커스터디 서비스 코퍼, 기업가치 2.61조 원으로 투자 유치 재개
코인데스크에 따르면 런던 소재 암호화폐 커스터디 및 거래 서비스 업체 코퍼(Copper)의 시리즈 C 투자 유치가 곧 마무리될 것으로 보인다. 기업은 지난해 30억 달러 기업 가치로 5억 달러 규모의 시리즈 C 라운드를 진행했으나, 영국 규제기관의 암호화폐 임시 운영 인가 이슈로 이를 연기했다. 현재 회사 가치는 약 20억 달러(2.61조 원)로 예상되며, 유치한 투자 규모는 알려지지 않았다. 업계 관계자는 "앞으로 몇 주 안에 마무리될 것"이라고 전했다.
날짜: 2022-07-14 02:47:00
*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: 3% | 악재: 0% | 중립: 97% ***
24시간동안 최고 상승치: +3.1737% | 24시간동안 최고 하락치: -3.1645%


Rank 2

디지털자산 서비스 업체 탑 글로벌, 영국 증권거래소 상장
코인텔레그래프에 따르면 영국 암호화자산 실시간 거래 서비스 업체 탑 글로벌(Tap Global)이 런던 Aquis 증권거래소 성장마켓(AQSE)에 상장됐다. 탑 글로벌 측은 "영국 거주가에게 규제 옵션을 제공하기 위해 영국 증권거래소 상장을 추진했다"고 설명했다.
날짜: 2023-01-11 06:09:00
*** 인공지능이 분석한 

# Example Run 5

In [ ]:
query_title="1시간 동안 2895억원 BTC 거래소 출금"
query_content="""크립토퀀트 알람에 따르면 1시간 동안 거래소에서 7,675.39 BTC (2,895억원)가 출금됐다. 커스터디나 OTC 거래를 지원하지 않는 거래소에서의 대량 출금은 신규 지갑이사일 수 있다.
1. Coinbase_pro 2,832 BTC (36%)
2. Binance 2,517 BTC (32%)
3. Kraken 1,213 BTC (15%)
"""

get_ranks(query_title, query_content) 

query news
1시간 동안 2895억원 BTC 거래소 출금
크립토퀀트 알람에 따르면 1시간 동안 거래소에서 7,675.39 BTC (2,895억원)가 출금됐다. 커스터디나 OTC 거래를 지원하지 않는 거래소에서의 대량 출금은 신규 지갑이사일 수 있다.
1. Coinbase_pro 2,832 BTC (36%)
2. Binance 2,517 BTC (32%)
3. Kraken 1,213 BTC (15%)

*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: 97% | 악재: 2% | 중립: 1% ***





Rank 1

지난 1시간 모든 거래소 밖으로 5,407억 원 규모 BTC 출금
암호화폐 마켓 데이터 분석 업체 크립토퀀트에 따르면, 지난 1시간 9,365.59 BTC (5,407억원)가 모든 거래소 밖으로 출금됐다. 커스터디나 OTC를 지원하지 않는 거래소의 경우 대량 입금은 신규 지갑 이사일 확률이 있다. 거래소별 입금 규모는 다음과 같다.
날짜: 2021-12-18 00:18:00
*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: 96% | 악재: 2% | 중립: 1% ***
24시간동안 최고 상승치: +3.1976% | 24시간동안 최고 하락치: -0.9231%


Rank 2

지난 1시간 모든 거래소 밖으로 4,167억 원 규모 BTC 출금
암호화폐 마켓 데이터 분석 업체 크립토퀀트에 따르면, 지난 1시간 7,339 BTC (약 4,167억 원)가 모든 거래소 밖으로 출금됐다. 커스터디나 OTC를 지원하지 않는 거래소의 경우 대량 입금은 신규 지갑 이사일 확률이 있다. 거래소별 입금 규모는 다음과 같다.
날짜: 2021-09-08 04:40:00
*** 인공지능이 분석한 비트코인 가격에 대한 영향: 호재: 96% | 악재: 2% | 중립: 1% ***
24시간동안 최고 상승치: +1.0077% | 24시간동안 최고 하락치: -4.3507%


Rank 3

지난 1시간 모든 거래소 밖으로 1조 원 규모 BTC 출금
암호